In [1]:
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver # 브라우저를 조작하는 모듈
from selenium.webdriver.chrome.service import Service # 크롬 드라이버의 시작과 중지를 담당하는 클래스
from selenium.webdriver.common.by import By # 엘리먼트를 어떤 방식으로 선택할 지에 대한 상수
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 관리
import pandas as pd
from tqdm.auto import tqdm
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

#### 저는 명시적 대기를 사용하였지만 암묵적 대기를 사용하시면 더 빠르게 실행되게 할 수도 있습니다.

In [2]:
DATA_PATH = "C:/Users/jjong/OneDrive/Desktop/project/data/"

In [3]:
def influence_searching(ID):
    url = "https://www.instagram.com/" + str(ID)
    return url

In [4]:
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, "div._aagw")[0]
    first.click()
    time.sleep(3)

In [5]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
    except:
        content = ''
    # 해시 태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    # 작성 일자
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    # 좋아요
    try:
        like = soup.select('section._ae5m._ae5n._ae5o > div > div > span > a > span')[0].text
        like = re.findall(r'[0-9]+', like)[0]
    except:
        like = 0
    
    # 댓글 더보기 클릭
    action = ActionChains(driver)
    for _ in range(100):
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button')))
            next_coment = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button')
            action.move_to_element(next_coment).perform()
            next_coment.click()
            driver.implicitly_wait(3)
        except:
            break
    # 아이디 및 댓글 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h3._a9zc")))
    try:
        username = [username.div.div.div.a.get_text() for username in soup.find_all('h3', {'class' :'_a9zc'})]
        comments = [coment.get_text() for coment in soup.find_all('span', {'class':'_aacl _aaco _aacu _aacx _aad7 _aade'})]
    except:
        username = ''
        comments = ''

    content = re.sub(r'#[^\s#,\\]+', "", content)
    data = [content, username, comments, date, like, tags]
    return data

In [6]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh')
    right.click()
    time.sleep(1.5)

In [7]:
# 인스타그램에 로그인
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get('https://www.instagram.com')
time.sleep(3)

email = str(' -- ') # 아이디 입력
input_id = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(1) > div > label > input').send_keys(email)

password = str(' -- ') # 페스워드 입력
input_pw = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(2) > div > label > input').send_keys(password)
time.sleep(1)

driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(3)").click()

In [8]:
baby_list = pd.read_csv(f'{DATA_PATH}list-baby.csv')
baby = baby_list['address']

In [14]:
id_list = []
for address in baby:
    id = address.split('/')[3]
    id_list.append(id)
id_list = id_list[28:45]
id_list

['do_ha0720',
 'minnie._.bbo',
 'yedamo_life',
 'twoyumom_life',
 'wooju_likeee',
 'onlmom._.ilbo',
 'joovely__',
 'chan_11.21',
 'yeajiiini',
 'ddmoon.bb',
 'ssong.stella',
 'luv_woong_happy',
 'prettyjiyoon.mom']

In [12]:
# 한명의 인플루언서마다 게시글 100개 크롤링
target = 100
for infuluence in tqdm(id_list, desc="outer", position=0):
    driver.get(f'https://www.instagram.com/{infuluence}/')
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span._ac2a")))
    count = int(driver.find_element(By.CSS_SELECTOR, "span._ac2a").text)
    
    driver.implicitly_wait(10)
    
    select_first(driver)
    
    results = []
    for j in tqdm(range(target), desc="inner", position=1):
        driver.implicitly_wait(10)
        if j >= count:
            break
        try:
            data = get_content(driver)
            results.append(data)
            try:
                move_next(driver)
            except:
                break
        except:
            time.sleep(1)
            try:
                move_next(driver)
            except:
                break

        time.sleep(1.5)
    
    results_df = pd.DataFrame(results)
    results_df.columns = ['content','username', 'comments', 'date', 'like', 'tags']
    results_df.to_csv(f'{DATA_PATH}_{infuluence}_{count}insta_crawling.csv', encoding='utf-8-sig')
    # 다음 ID로 넘어가기전 너무 빠르게 하면 오류 및 사람인지 확인하는 메시지가 떠서 저는 우선 30초를 줬습니다. 
    # 본인이 해보시고 오류가 안나는 선에서 시간을 정하시면 됩니다.
    
    time.sleep(30)
driver.quit()

outer:   0%|          | 0/13 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

...

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
# 오류났을때
results_df

,content,username,comments,date,like,tags
0,421일♡하 진짜 너무 사랑스럽자나..🙈토닥토닥해주던 엄마대신해서 셀프재우기🥰,"[hyonlub, joy_and_l, imji_wonss, sojeonggim10,...","[ㅇㅓ머ㅠㅠㅠ 넘 이뻐요...!!!, 아 너무 사랑스로미😍 많이 받은 사랑을 다른 ...",2023-06-08,1316,[]
1,"야시장에서 인형타기 게임할때마다 드는생각""아 이돈으로 닭꼬치나 하나 더 사먹을껄...😔”","[eunmira6, ippdaji, hjheart6, doramafaldarevez...","[ㅋㅋㅋㅋ진짜 아기랑 잘놀아주시네요., 아빠가 더 신나셨네요~~~, 아버님이 더 ...",2022-12-08,4958,[]
2,𝚢𝚒𝚜𝚎𝚘ෆ +𝟾𝟸𝚍𝚊𝚢개월수에비해 진짜 많이데리고 나간것같은데 카페가본건 처음인것...,"[sktcy90, newmmeme, bebe_.recipe, 202.2318, an...","[꺄 너무 귀여워요, 하이오즈에 애기수트안보이는데 어떤걸까요? 옷너무귀여워요, 너무...",2022-07-04,1,[]
3,497일♡더운날 둘이 카페데이트라도 하고싶어도 아기의자는 거부하고ㅜㅜ잡으러다니느라 ...,"[k.minse0, 2seo_home]","[ㅎㅎ언니도 오늘 여기 가려고했는뒈!!!!!! 여기 가봐야겠다 ㅎㅎ😍, 이서사랑해여🧡]",2023-08-23,124,"[#웰컴키즈존, #아기랑카페]"
4,495일♡쓰레기버릴겸 산책나갔다가 크게 혼쭐나고 돌아옴밖은 날씨가 정말살벌해.. 집...,"[goodi__mj, writtenbysey, ssu__nny, chachamom_...",[집에서도 웃으면서 잔놀아주는 착한아기야❤️5개월차 아들맘입니다.☺️소통하며 지내용...,2023-08-21,417,"[#집콕놀이, #싱크대놀이]"
...,...,...,...,...,...,...
95,슬기로운 격리생활 그래도 웃을일은 매일 생기네요🤣좋은기억 심어줘서 약잘먹는 아기로 ...,"[with.u.yeseo, adongaaa, 2seo_home, heejuuu_k,...","[노선을 살짝 변경했구만요~~~♡, 갑자기 바쁜일😂❤️ 이서야 고생이 많아아❤️ 언...",2022-12-29,441,"[#코로나패밀리, #왜아직도목요일]"
96,이서네 근황남편에이어 결국 이서도 확진아픈와중에 저렇게 웃어주면 나는 정말 눙무리가...,"[sung2hyun, jihye.jiyu__, yeonju_1.23, love22_...","[하ㅠㅠ 안쓰러워요 ㅠㅠ정말 이서야 ㅜㅜㅜ 조용히지나가길빌게요 ㅠㅠ, 저희랑 반대예...",2022-12-27,858,[]
97,오붓하게 조촐하게 크리스마스 끄읕🎄선물도많이받구 이서두 행복했떠?💕내년엔 집콕말고 ...,"[adongaaa, 2seo_home, harami__yun, seo_mjmj]","[어디가 조촐하게죠오~~?😍❤️이서도 선물도 받고 너무 좋겠다아❤️, 256일🧡, ...",2022-12-25,422,[]
98,𝚖𝚊𝚛𝚛𝚢 𝚌𝚑𝚛𝚒𝚜𝚝𝚖𝚊𝚜 𝚢𝚒𝚜𝚎𝚘❤︎,"[adongaaa, 22.joon__, little_coya_05, 2seo_hom...","[이서야 메리크리스마스야아❤️😍, 귀요미 루돌푸 이서야 행복한 메리크스마스 ! 🎄🎄...",2022-12-24,384,[#하필팅팅부은날🤣]


In [19]:
results_df.to_csv(f'{DATA_PATH}크롤링02/_{infuluence}_{count}insta_crawling.csv', encoding='utf-8-sig')